In [1]:
import tensorflow as tf
tf.enable_eager_execution()
from exjobb_data import *

Using TensorFlow backend.


exjobb data prepare loading...   
data loaded ....


In [2]:
with open('concated_cars.pkl','rb') as f:
    concated_cars = pickle.load(f)
    print(concated_cars.shape)

with open('t_encoded_cars.pkl','rb') as f:
    t_encoded_cars = pickle.load(f)
    print(t_encoded_cars.shape)

(29061, 35)
(9082, 35)


In [3]:
with open('np_ctxs.pkl','rb') as f:
    np_ctxs = pickle.load(f)
with open('np_t_ctxs.pkl','rb') as f:
    np_t_ctxs = pickle.load(f)

In [4]:
ctx_df = pd.DataFrame(np_ctxs)
np_ctxs = ctx_df[user_columns].values

In [5]:
np_ctxs.shape

(29061, 54)

In [6]:
np_ctxs = np_ctxs.astype('float32')
np_t_ctxs = np_t_ctxs.astype('float32')

In [7]:
with open('np_rs.pkl','rb') as f:
    np_rs = pickle.load(f)
    print(np_rs.shape)
with open('np_t_rs.pkl','rb') as f:
    np_t_rs = pickle.load(f)
    print(np_t_rs.shape)

(29061,)
(9082,)


In [8]:
beta_scaler = MinMaxScaler()
beta_scaler.fit(concated_cars)
concated_cars = beta_scaler.transform(concated_cars)
t_concated_cars = beta_scaler.transform(t_encoded_cars)

In [ ]:
alpha_scaler = MinMaxScaler()
alpha_scaler.fit(np_ctxs)
before_ =np_ctxs
np_ctxs = alpha_scaler.transform(np_ctxs)
np_t_ctxs = alpha_scaler.transform(np_t_ctxs)

## SVDFeature algorithm implementation

In [ ]:
from sklearn.base import BaseEstimator

import numpy as np
class SVDFeature(BaseEstimator):
    def __init__(self,size_gamma,size_alpha,size_beta,hidden_dim=200, lr=0.002):
        self.size_gamma = size_gamma
        self.size_alpha = size_alpha
        self.size_beta = size_beta
        self.hidden_dim = hidden_dim
        self.lr = tf.constant(lr,dtype=tf.float32)
        self.mu = 0
        self.mu_ittr = 0 
        #init
        self.p = tf.random.normal((size_alpha, hidden_dim),0,0.1)
        self.q = tf.random.normal((size_beta, hidden_dim),0,0.1)
        print('init p shape',self.p.shape)
        print('init q shape',self.q.shape)
        
        self.alpha_bias = tf.zeros((size_alpha,1),dtype=tf.float32)
        self.beta_bias = tf.zeros((size_beta,1),dtype=tf.float32)
        self.gamma_bias = tf.zeros((size_gamma,1),dtype=tf.float32)
        print(self.gamma_bias.shape)
        self.fitted_betas=[]
        self.fitted_indexes=[]

    def predict(self,gamma,alpha,beta,reshape=False):

        if reshape:
            beta = tf.reshape(beta,(-1,1))
            print(beta.shape)
        assert self.gamma_bias.shape == gamma.shape
        assert self.alpha_bias.shape == alpha.shape
        assert self.beta_bias.shape == beta.shape

        gamma_bias = tf.matmul(tf.transpose(self.gamma_bias),gamma)
        alpha_bias = tf.matmul(tf.transpose(self.alpha_bias),alpha)
        beta_bias = tf.matmul(tf.transpose(self.beta_bias),beta)
        
        temp_p = tf.matmul(tf.transpose(self.p),alpha)
        temp_q = tf.matmul(tf.transpose(self.q),beta)
  
        p_q = tf.matmul(tf.transpose(temp_p),temp_q)

        y = (gamma_bias+alpha_bias+beta_bias)+p_q

        return tf.sigmoid(y)
    
    def predict_all(self,x):
        max_p=0
        max_y = None
        max_idx = 0
        random_ys = np.random.randint(0,len(self.fitted_betas)-1,1000)
        for y_idx in random_ys:
            y = self.fitted_betas[y_idx]
            prediction = self.predict(tf.zeros(self.gamma_bias.shape),x,y)
            if prediction > max_p:
                max_p = prediction
                max_y = y
                max_idx = y_idx
        return y,max_idx

    def predict_all_vector(self,contexts):
        ct_p = tf.matmul(contexts,svdfeature.p)
        return tf.matmul(ct_p,tf.transpose(svdfeature.q))
        
    @tf.contrib.eager.defun
    def top_k_ys(self,x,ys,top_k=5):
        lambda_fn = lambda y: self.predict(tf.zeros(self.gamma_bias.shape),x,y)
        res_tens = tf.map_fn(lambda_fn,ys)
        return tf.math.top_k(tf.reshape(res_tens, (-1,)),top_k)

    
    def backward(self, e, gamma, alpha, beta):
        q_beta = tf.transpose(tf.matmul(tf.transpose(self.q),beta))
        p_update = self.p + self.lr*(
            tf.multiply(
                tf.multiply(e,alpha),
                tf.transpose(tf.matmul(tf.transpose(self.q),beta))
            )-0*tf.norm(self.p)*self.p)
        
        q_update = self.q + self.lr*(
            tf.multiply(
                tf.multiply(e,beta),
                tf.transpose(tf.matmul(tf.transpose(self.p),alpha))
            )-0*tf.norm(self.q)*self.q)

        assert p_update.shape == self.p.shape
        assert q_update.shape == self.q.shape
        self.q = q_update
        self.p = p_update
        #print('p updated', self.p)
        self.beta_bias = self.beta_bias + self.lr*(tf.multiply(e, beta)-0*tf.norm(self.beta_bias)*self.beta_bias)
        self.alpha_bias = self.alpha_bias + self.lr*(tf.multiply(e, alpha)-0*tf.norm(self.alpha_bias)*self.alpha_bias)
        self.gamma_bias = self.gamma_bias + self.lr*(tf.multiply(e, gamma)-0*tf.norm(self.gamma_bias)*self.alpha_bias)
        
    def fit(self,a,b,r,**fit_params):
        assert a.shape[0] == b.shape[0]
        assert r.shape[0] == a.shape[0]
        assert a.shape[1] == self.size_alpha
        assert b.shape[1] == self.size_beta
        
        #random training items
        self.trained = 0
        losses = []
        trained=0
        with tf.GradientTape() as t:
            predictions = []
            actuals = []
            for epoch in tqdm_notebook(range(20)):
                idxes = np.random.randint(0,a.shape[0]-1,size=1000)
                self.fitted_indexes+=idxes.tolist()
                e_predictions = []
                e_actuals = []
                for idx in tqdm_notebook(idxes):
                    #print('index',idx)
                    gamma =tf.zeros(self.gamma_bias.shape)
                    a_sample = tf.reshape(a[idx],self.alpha_bias.shape)
                    b_sample = tf.reshape(b[idx],self.beta_bias.shape)
                    self.fitted_betas += [b_sample]
                    y_pred = self.predict(gamma, a_sample, b_sample)
                    y_pred = y_pred[0][0]
                    if len(e_predictions) % 1000 == 0:
                        print('ypred',y_pred)
                        print('avarage error', self.loss(e_actuals,e_predictions))

                    e_predictions += [y_pred.numpy()]
                    self.mu += y_pred.numpy()
                    self.mu_ittr += 1
                    e_actuals += [r[idx]]
                    e = self.error(r[idx],y_pred)
                    if len(e_predictions) % 1000 == 0:
                        print('error',e)

                    self.backward(e,gamma,a_sample,b_sample)
                predictions += [e_predictions]
                actuals += [e_actuals]
                print('avarage error', self.loss(e_actuals,e_predictions))
        self.fitted_betas = np.array(self.fitted_betas)

    def find_max_scoring(self,X,ys):
        all_scores = self.predict_all(X,ys)
        argmax_score = tf.math.argmax(all_scores)
        return ys[argmax_score.numpy()[0][0]]
            
    def error(self,y_true,y_pred):
        return (y_true-y_pred)
    
    def calc_loss(self,X,y):
        x_sample = tf.reshape(X,self.alpha_bias.shape)
        y_sample = tf.reshape(y,self.beta_bias.shape)
        return self.loss(1, self.predict(tf.zeros(self.gamma_bias.shape), x_sample, y_sample))
    
    @tf.contrib.eager.defun
    def loss_all(y_trues,y_preds):
        lambda_fn = lambda y_t,y_p: self.loss(y_t,y_p)
        res_tens = tf.map_fn(lambda_fn,y_trues,y_preds)
        return tf.math.reduce_mean(res_tens)
    
    def loss(self, y_true, y_pred):
        return tf.losses.mean_squared_error(y_true,y_pred)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [ ]:
svdfeature = SVDFeature(1,np_ctxs.shape[1],concated_cars.shape[1])

init p shape (54, 200)
init q shape (35, 200)
(1, 1)


In [ ]:
svdfeature.fit(np_ctxs,concated_cars,np_rs,lr=0.001)

ypred tf.Tensor(0.4703475, shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.006914258, shape=(), dtype=float32)
avarage error tf.Tensor(0.013261027, shape=(), dtype=float32)


ypred tf.Tensor(0.99104124, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.0013821125, shape=(), dtype=float32)
avarage error tf.Tensor(0.00065906433, shape=(), dtype=float32)


ypred tf.Tensor(0.9541363, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.0029950738, shape=(), dtype=float32)
avarage error tf.Tensor(0.00028180203, shape=(), dtype=float32)


ypred tf.Tensor(0.9981634, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.021552503, shape=(), dtype=float32)
avarage error tf.Tensor(0.00016003933, shape=(), dtype=float32)


ypred tf.Tensor(0.9998135, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.001080513, shape=(), dtype=float32)
avarage error tf.Tensor(0.0001364051, shape=(), dtype=float32)


...

ypred tf.Tensor(0.9998716, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.00045531988, shape=(), dtype=float32)
avarage error tf.Tensor(3.3774246e-05, shape=(), dtype=float32)


ypred tf.Tensor(0.9999962, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.00012385845, shape=(), dtype=float32)
avarage error tf.Tensor(2.1811189e-05, shape=(), dtype=float32)


ypred tf.Tensor(0.99999475, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(9.179115e-05, shape=(), dtype=float32)
avarage error tf.Tensor(4.5558085e-05, shape=(), dtype=float32)


ypred tf.Tensor(0.99997246, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(2.3841858e-07, shape=(), dtype=float32)
avarage error tf.Tensor(2.0692658e-05, shape=(), dtype=float32)


ypred tf.Tensor(0.99914837, shape=(), dtype=float32)
avarage error tf.Tensor(0.0, shape=(), dtype=float32)
error tf.Tensor(0.00036799908, shape=(), dtype=float32)
avarage error tf.Tensor(1.4417869e-05, shape=(), dtype=float32)



In [ ]:
predicted_cars =[]
predicted_cars = beta_scaler.inverse_transform(svdfeature.predict_all_vector(np_t_ctxs))
with open('predicted_features.pkl','wb') as f:
    pickle.dump(predicted_cars, f)
